In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/equity-post-HCT-survival-predictions/sample_submission.csv
/kaggle/input/equity-post-HCT-survival-predictions/data_dictionary.csv
/kaggle/input/equity-post-HCT-survival-predictions/train.csv
/kaggle/input/equity-post-HCT-survival-predictions/test.csv
/kaggle/input/dataset2/ensemble_models.pkl


In [2]:
import numpy as np
import pandas as pd
import joblib
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor

# Load Test Data
test = pd.read_csv("/kaggle/input/equity-post-HCT-survival-predictions/test.csv")

# Preprocess Test Data (Same as Training)
def feature_engineering(df):
    df['donor_age_diff'] = df['donor_age'] - df['age_at_hct']
    df['score_interaction'] = df['comorbidity_score'] * df['karnofsky_score']
    df['age_group'] = df['age_at_hct'] // 10
    df['dri_score_NA'] = df['dri_score'].apply(lambda x: int('N/A' in str(x)))
    return df

test = feature_engineering(test)

# Encode categorical features (Same as Training)
for col in test.select_dtypes(include=['object', 'category']).columns:
    test[col] = test[col].astype('category').cat.codes

# Load Trained Models
lgb_models, cat_models = joblib.load("/kaggle/input/dataset2/ensemble_models.pkl")

# Ensure Test Data Has the Same Features as Training
train_features = set(lgb_models[0].feature_name_)  # Get feature names from trained LightGBM model
test_features = set(test.columns)

missing_in_test = train_features - test_features
extra_in_test = test_features - train_features

# Add missing columns with default values
for col in missing_in_test:
    if col == "weight":
        test[col] = 1.0  # Default weight value
    else:
        test[col] = 0


# Ensure column order matches training
test = test[list(train_features)]

# Make Predictions
lgb_preds = np.mean([model.predict(test) for model in lgb_models], axis=0)
cat_preds = np.mean([model.predict(test) for model in cat_models], axis=0)

# Weighted Ensemble (60% LightGBM + 40% CatBoost)
final_preds = 0.6 * lgb_preds + 0.4 * cat_preds

# Save Submission File
submission = pd.DataFrame({
    "ID": test["ID"],  # Ensure test dataset has 'ID' column
    "prediction": final_preds
})
submission.to_csv("submission.csv", index=False)

print("Inference complete! Submission file saved as submission.csv")

Inference complete! Submission file saved as submission.csv
